<a href="https://colab.research.google.com/github/duongquangvinh/Fundamental-Machine-Learning-model/blob/main/keras_sequential_dogcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam
import os
import random
import shutil

## Preprocess Data

In [ ]:
!unzip drive/MyDrive/Dataset/dog_cat.zip

In [10]:
data_dir = "./dog_cat"
dog = []
cat = []
for img in os.listdir(data_dir):
    label = img.split(".")[0]
    img_path = os.path.join(data_dir, img)
    if label == "dog":
        dog.append(img_path)
    else:
        cat.append(img_path)

rate = 0.2
test_cat = random.sample(cat, int(rate*len(cat)))
test_dog = random.sample(dog, int(rate*len(dog)))

train_cat = list(set(cat) ^ set(test_cat))
train_dog = list(set(dog) ^ set(test_dog))

save_dir = "./data/"
for test_cat_img, test_dog_img in zip(test_cat, test_dog):
    img_cat_name = test_cat_img.split("/")[-1]
    img_dog_name = test_dog_img.split("/")[-1]

    save_test_cat = os.path.join(save_dir,"test","cat")
    os.makedirs(save_test_cat, exist_ok=True)
    save_test_dog = os.path.join(save_dir,"test","dog")
    os.makedirs(save_test_dog, exist_ok=True)

    shutil.copy2(test_cat_img, os.path.join(save_test_cat, img_cat_name))
    shutil.copy2(test_dog_img, os.path.join(save_test_dog, img_dog_name))

for train_cat_img, train_dog_img in zip(train_cat, train_dog):
    img_cat_name = train_cat_img.split("/")[-1]
    img_dog_name = train_dog_img.split("/")[-1]

    save_train_cat = os.path.join(save_dir,"train","cat")
    os.makedirs(save_train_cat, exist_ok=True)
    save_train_dog = os.path.join(save_dir,"train","dog")
    os.makedirs(save_train_dog, exist_ok=True)

    shutil.copy2(train_cat_img, os.path.join(save_train_cat, img_cat_name))
    shutil.copy2(train_dog_img, os.path.join(save_train_dog, img_dog_name))

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_set = train_datagen.flow_from_directory('data/train',
                                                target_size=(200, 200),
                                                batch_size=32,
                                                class_mode='binary', shuffle=True)
test_set = test_datagen.flow_from_directory('data/test',
                                                target_size=(200, 200),
                                                batch_size=32,
                                                class_mode='binary',shuffle=False)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [6]:
class LeNet(Sequential):
    def __init__(self):
        super(LeNet, self).__init__()
        self.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(200, 200, 3), padding="same"))
        self.add(MaxPool2D(strides=2))
        self.add(Conv2D(filters=48, kernel_size=(5, 5), activation='relu', padding='valid'))
        self.add(MaxPool2D(strides=2))
        self.add(Flatten())
        self.add(Dense(256, activation='relu'))
        self.add(Dense(84, activation='relu'))
        self.add(Dense(1, activation='softmax'))

        self.compile(optimizer=Adam(learning_rate=5e-4),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [9]:
net = LeNet()
history = net.fit(train_set, steps_per_epoch=(20000//32),epochs= 1,
                             validation_data = test_set, validation_steps= (5000//32),
                            verbose=1)

checkpoint_dir = "./checkpoint/"
os.makedirs(checkpoint_dir, exist_ok=True)

net.save_weights(os.path.join(checkpoint_dir, "keras_cifar10.h5"))

# evaluate model and save
_, acc = net.evaluate(test_set,steps = (5000//32), verbose = 0)
print(f"Accuracy: {acc*100}")

625/625 [==============================] - 2320s 4s/step - loss: 0.6196 - accuracy: 0.5000 - val_loss: 0.5628 - val_accuracy: 0.4992
Accuracy: 49.919870495796204
